In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
print(np.__version__)

1.26.4


In [4]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [5]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [6]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [7]:
glove_wiki_vectors.most_similar("soccer")

[('football', 0.8732220530509949),
 ('basketball', 0.8003970980644226),
 ('volleyball', 0.7690008878707886),
 ('hockey', 0.7440757751464844),
 ('league', 0.7424143552780151),
 ('sports', 0.7309418320655823),
 ('club', 0.7297094464302063),
 ('rugby', 0.7242490649223328),
 ('team', 0.7151049375534058),
 ('tennis', 0.7036359310150146)]

In [8]:
glove_wiki_vectors.most_similar("computer")

[('computers', 0.875198245048523),
 ('software', 0.8373122215270996),
 ('technology', 0.7642159461975098),
 ('pc', 0.7366448640823364),
 ('hardware', 0.7290391325950623),
 ('internet', 0.72867751121521),
 ('desktop', 0.7234441637992859),
 ('electronic', 0.7221828699111938),
 ('systems', 0.7197922468185425),
 ('computing', 0.7141730189323425)]

In [9]:
glove_wiki_vectors.most_similar("airplane")

[('plane', 0.8651753664016724),
 ('jet', 0.8155897259712219),
 ('aircraft', 0.7861484289169312),
 ('airplanes', 0.7662243843078613),
 ('airliner', 0.7646356225013733),
 ('jetliner', 0.7469576597213745),
 ('flight', 0.7339501976966858),
 ('planes', 0.7338765263557434),
 ('flying', 0.7300053834915161),
 ('crash', 0.7099080085754395)]

In [10]:
glove_wiki_vectors.most_similar("travel")

[('travelers', 0.7476776838302612),
 ('trips', 0.7453446388244629),
 ('traveling', 0.7360444068908691),
 ('trip', 0.7348803877830505),
 ('flights', 0.6884056925773621),
 ('vacation', 0.6829158663749695),
 ('destinations', 0.6691367030143738),
 ('destination', 0.6632232666015625),
 ('visit', 0.6496120095252991),
 ('business', 0.6473838090896606)]

In [11]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [12]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [13]:
# from lecture 18
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1]))
    )

The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [14]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [15]:
for word in test_words:
    print(f"{word} is in vocabulary" if word in glove_wiki_vectors else f"{word} is not in vocabulary")

covididiot is not in vocabulary
fomo is not in vocabulary
frenemies is in vocabulary
anthropause is not in vocabulary
photobomb is not in vocabulary
selfie is not in vocabulary
pxg is not in vocabulary
pacg is not in vocabulary
cct is in vocabulary
escc is in vocabulary


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [16]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [17]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [18]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.1428324

In [19]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [20]:
analogy("young", "cool", "old")

young : cool :: old : ?


,Analogy word,Score
0,dry,0.627798
1,warm,0.603297
2,hot,0.586539
3,cold,0.574672
4,cooling,0.561285
5,refrigerator,0.551792
6,cooler,0.547265
7,wet,0.539563
8,fashioned,0.526971
9,soak,0.526891


In [21]:
glove_wiki_vectors.similarity("burger", "delicious")

0.4155323

In [22]:
glove_wiki_vectors.similarity("pizza", "delicious")

0.551952

In [23]:
analogy("extrovert", "social", "introvert")

extrovert : social :: introvert : ?


,Analogy word,Score
0,political,0.640161
1,education,0.626748
2,issues,0.623314
3,welfare,0.611575
4,problems,0.609818
5,health,0.603912
6,economic,0.591203
7,society,0.589185
8,reform,0.587446
9,politics,0.584211


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

There are some worrisome biases in these embeddings. Let's look at the examples that I came up with. The embeddings are biased towards the impression that the young people are cool and that old people are dry, which is clearly false. Similarity tests showed that the model correlate pizza to deliciousness more than compared to burger, which is definitely biased. Finally, the embeddings connect introverts to political as extroverts to social, which is inaccurate as well.

Using embeddings with biases in our models can **further consolidate false stereotypes into people's minds, making people think that these stereotypes are in fact accurate**. This is something that we want to avoid as responsible ML engineers. Our models should provide results that are as accurate to the real world context as possible, and having these hidden biases in our embeddings that we rely on so much is a problem.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [24]:
from sklearn.datasets import fetch_20newsgroups

In [25]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [26]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

#### We can separate the texts into approximately 4 topics - computers, sports, religion, and politics...

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [27]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

Write code using spaCy to preprocess the text column in the given dataframe df and save the processed text in a new column called text_pp within the same dataframe.

In [28]:
df.head()

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast


In [29]:
import re
# FROM LECTURE 18

def preprocess_spacy(text):
    # Replace URL, EMAIL, NUM
    text = re.sub(r"http\S+|www\S+", "URL", text)
    text = re.sub(r"\S+@\S+", "EMAIL", text)
    text = re.sub(r"\b\d+\b", "NUM", text)
    doc = nlp(text)
    res = []
    for token in doc:
        # Remove stop words and punctuation
        if not token.is_stop and not token.is_punct and token.is_alpha:
            res.append(token.lemma_.lower()) # Perform lemmatization here
    return " ".join(res)

In [30]:
# # FROM LECTURE 18
# # Run only once, then just read from the saved csv
# df["text_pp"] = [preprocess_spacy(text) for text in df["text"]]
# df.to_csv("data/df_pp.csv")

In [31]:
df_processed = pd.read_csv("data/df_pp.csv")

In [32]:
df_processed.head()

,Unnamed: 0,text,target,target_name,text_pp
0,0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read num num sence wonder help num num pr...
1,1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

- replaced URLs, emails, and numbers with their generic tokens as mentioned in the criteria with regex
- tokenized regex-processed sentences into words and lemmatized the ones that are not stopwords (words like `is`, `are`, `and` that have no intrinsic meaning), punctuation, and non-alphabetical characters such as whitespaces. I'm doing this to make sure that our processed text only carries the essential information without too much noise as efficiently as possible.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [34]:
df_processed["text_pp"].head()

0    know read num num sence wonder help num num pr...
1    bad question ref list algorithm think bit hard...
2    actuallay hand support idea have newsgroup asp...
3    follow problem bug appreciate help create wind...
4    late upi foreign ministry spokesman ferhat ata...
Name: text_pp, dtype: object

In [35]:
# FROM LECTURE 18

from sklearn.feature_extraction.text import CountVectorizer

drop_df = df_processed.dropna(subset=["text_pp"]) 
vec = CountVectorizer(stop_words="english")
df_X = vec.fit_transform(drop_df["text_pp"])
df_X

<4450x30686 sparse matrix of type '<class 'numpy.int64'>'
	with 263930 stored elements in Compressed Sparse Row format>

In [36]:
# let's see our vocabulary constructed from the preprocessed texts
vocab = vec.get_feature_names_out()
vocab

array(['aa', 'aaa',
       'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg',
       ..., 'zyxel', 'zzzzzz', 'zzzzzzt'], dtype=object)

In [37]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4 # number of topics
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)

In [38]:
document_topics = lda.fit_transform(df_X)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [39]:
# from lecture 18
import mglearn

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)


topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           god           num           num           
people        people        email         game          
gun           know          file          year          
armenian      think         program       team          
armenians     num           use           think         
israel        believe       image         know          
kill          jesus         window        good          
state         thing         include       say           
government    say           available     play          
turkish       like          run           like          




I would say that the 4 topics are pretty easy to interpret and identify...

- Topic 0: Politics (regarding wars and conflicts, especially in the Middle East)
- Topic 1: Religion
- Topic 2: Computers (Info-tech related)
- Topic 3: Sports

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [40]:
topics = ["Politics", "Religion", "Computers", "Sports"]

data = {"Text": [], "Topic": []}

for text in list(df_processed['text_pp'].head(5)):
    data['Text'].append(text)

for probs in list(document_topics[0:5]):
    probs = list(probs)
    topic_index = probs.index(max(probs))
    data["Topic"].append(topics[topic_index])
    
pd.set_option('display.max_colwidth', None)  # Show full content of each column

topics_df = pd.DataFrame(data)
topics_df

,Text,Topic
0,know read num num sence wonder help num num provide paragraph num shall unlawful person transfer possess machinegun get look law dictionary find person artificial entity create government right federal constitution understand statute like num enforce individual tell government tell possess pass law law know law constitional go court go court run mill ok guilty jail claim right possess tell supreme court right,Politics
1,bad question ref list algorithm think bit hard num point find plane define circle lie algorithm num find center circle line pass center perpendicular plane point pass center sphere num repeat unused point original point give different line pass sphere origin interection center sphere num radius easy compute distance center original point leave math workable algorithm alternate method pair point plane form perpendicular bisector line segment pair contain center sphere pair form plane intersect point easy implement,Computers
2,actuallay hand support idea have newsgroup aspect graphic programming propose brian reply original posting suggest loose structure like reason make group possible reason spilt posting day take time browse posting find interested understand agree want aspect graphic meeting agree extension news forum exchange idea help help think difficult achive different thing meeting good evening,Religion
3,follow problem bug appreciate help create window child root child keypress event discard buttonpress event fall subwindow set fyi xnew olvwm fundamentally wrong,Religion
4,late upi foreign ministry spokesman ferhat ataman tell journalist turkey close air space flight armenia prevent humanitarian aid reach republic overland turkish territory historically uncivilized people exhibit sign compassion allow humanitarian aid reach civilian population nazis turkey publicly pronounce hypocrite choose continue condemnation serbians,Politics


I would say that the the first, second and the last documents are categorized to the correct topic. However, we can see that the third and the fourth documents clearly belong to the topic `Computers`, but were assigned into `Religion`. Even though we cannot see from such a small sample size hoe the document topic assignment does on the entire data set, we can already infer that it is not entirely accurate.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering is the process of making a profile for each user with the items that the user has already rated. We would need the characteristics or features for the item type, with which we are going to use to train a Regression model for each user that predict the rating the user gives each item. Once we have a regression model for each user, given any item and its features, we should be able to predict the rating that the user will give to the item.
2. One of the negative consequences of recommender systems is creating filter bubbles, which narrows a user’s exposure to diverse viewpoints and information by exposing only items that the user will very possibly like instead of the offering variety. Another potential drawback of the recommender system is that building one requires taking personal information about the user, which can raise security and privacy concerns.
3. Transfer learning is the process where instead of training a model from scratch, we leverage knowledge gained from a large general purposed data set (such as Wikipedia) to improve performance on another related but more specific task (e.g. sentiment analysis). Basically, we have pre-trained models that already knows the semantics and basic grammar of the language, and we use this model as a foundation on which to build a more niche language model targeted towards a more specific language problem by fine tuning the model with additional data.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)